# Introduction

In this notebook, I will get current ratings for 50,000 Lichess players and save them to my local player-opening database.

## The Problem

Ope! I did a silly. I forgot to get the ratings of all 50,000 lichess players in my database.

## The Solution

Luckily, this is an easy problem to solve. Here are the steps:

### 1. Database

- Update the database schema in db_utils to take a `rating` integer on the player table
- Note that this database has already been made so we're not actually changing anything immediately by doing this.  But if the schema is ever used to duplicate or remake the DB, we'll want that rating in there.

### 2. Update existing DB
- In step 1, we updated our schema but that'll only add `rating` if the schema is run again to make a new DB
- So, in this notebook, we'll add a simple integer `rating` column to `player`

### 3. Get player ratings

1. Get all 50,000 player usernames from our DB
2. In batches of 300 usernames, call POST lichess.org/api/users with my lichess auth token (env LICHESS_TOKEN), with the usernames in an array
    - See notebook 15's lichess.org api call for structure
3. For each batch, do a bulk update of usernames in the DB.

NOTE ON API LIMITS:
- We can only get max 8,000 users every ten minutes, and max 120,000 users per day
- The 120k limit shouldn't be an issue since we only have 50k players
- But we'll need simple timeouts and backoffs. Two requests per minute (of 300 players each) seems sensible; that's 6,000 players per ten minutes, well within the limit.
- So this will take about 90-120 minutes to run, which is fine.

## Rating specs:

We use blitz, rapid and classical games for our data. So there's some question about which rating to use. We'll employ the following formula and specs:

- Blitz is the most common time control in our dataset, so we'll prefer blitz.
- If rating deviation (RD) is <110, use blitz rating
- If blitz RD is >110 and Rapid RD more than 20 points below Blitz RD, use Rapid rating
- Only use Classical rating if both Blitz and Rapid are more than 110, and Classical is less than 110
- If all else fails, use Blitz rating
- The player will always have at least one of these ratings because we're only using highly active players, but if somehow they don't, use NAN or something else that makes sense
- We'll make a list of every player who gets listed as NAN and print them at the end

## Variables

At the top we'll define helpful variables that might need tweaking, including:
- How many lichess API calls to run before stopping
- How many max players to do total
- backoff times (default to two calls every one minute)

## Timing:

We'll include logging that is fairly detailed about the number of batches done so far, number of players remaining, number of NAN ratings, and ETA for the rest of the job.



## API response example

A single user from the lichess API looks like this:

[
  {
    "id": "thibault",
    "username": "thibault",
    "perfs": {
      "ultraBullet": {
        "games": 3,
        "rating": 1688,
        "rd": 350,
        "prog": 0,
        "prov": true
      },
      "bullet": {
        "games": 7475,
        "rating": 1787,
        "rd": 78,
        "prog": -6
      },
      "blitz": {
        "games": 11536,
        "rating": 1778,
        "rd": 47,
        "prog": 3
      },
      "rapid": {
        "games": 873,
        "rating": 1746,
        "rd": 131,
        "prog": -71,
        "prov": true
      },
      "classical": {
        "games": 24,
        "rating": 1806,
        "rd": 246,
        "prog": -5,
        "prov": true
      },
      "correspondence": {
        "games": 377,
        "rating": 1942,
        "rd": 148,
        "prog": -12,
        "prov": true
      },
      "chess960": {
        "games": 348,
        "rating": 1551,
        "rd": 254,
        "prog": 61,
        "prov": true
      },
      "kingOfTheHill": {
        "games": 94,
        "rating": 1744,
        "rd": 275,
        "prog": 14,
        "prov": true
      },
      "threeCheck": {
        "games": 66,
        "rating": 1728,
        "rd": 246,
        "prog": 132,
        "prov": true
      },
      "antichess": {
        "games": 72,
        "rating": 1512,
        "rd": 272,
        "prog": -20,
        "prov": true
      },
      "atomic": {
        "games": 99,
        "rating": 1633,
        "rd": 288,
        "prog": 18,
        "prov": true
      },
      "horde": {
        "games": 46,
        "rating": 1592,
        "rd": 268,
        "prog": -20,
        "prov": true
      },
      "racingKings": {
        "games": 13,
        "rating": 1552,
        "rd": 320,
        "prog": -75,
        "prov": true
      },
      "crazyhouse": {
        "games": 50,
        "rating": 1567,
        "rd": 286,
        "prog": -34,
        "prov": true
      },
      "puzzle": {
        "games": 5696,
        "rating": 1954,
        "rd": 71,
        "prog": 0
      },
      "storm": {
        "runs": 44,
        "score": 33
      },
      "racer": {
        "runs": 82,
        "score": 51
      },
      "streak": {
        "runs": 49,
        "score": 33
      }
    },
    "flair": "nature.seedling",
    "patron": true,
    "patronColor": 9,
    "verified": true,
    "createdAt": 1290415680000,
    "profile": {
      "bio": "I turn coffee into bugs.",
      "realName": "Thibault Duplessis",
      "links": "github.com/ornicar\r\nmas.to/@thibault"
    },
    "seenAt": 1758635849909,
    "playTime": {
      "total": 6436845,
      "tv": 17974
    }
  },
  {
    "id": "maia1",
    "username": "maia1",
    "perfs": {
      "bullet": {
        "games": 193064,
        "rating": 1512,
        "rd": 45,
        "prog": 17
      },
      "blitz": {
        "games": 357366,
        "rating": 1412,
        "rd": 45,
        "prog": -12
      },
      "rapid": {
        "games": 408246,
        "rating": 1590,
        "rd": 45,
        "prog": -25
      },
      "classical": {
        "games": 118909,
        "rating": 1626,
        "rd": 45,
        "prog": 10
      },
      "correspondence": {
        "games": 265,
        "rating": 1527,
        "rd": 206,
        "prog": -18,
        "prov": true
      }
    },
    "title": "BOT",
    "verified": true,
    "createdAt": 1582579972726,
    "profile": {
      "bio": "Maia is a human-like neural network chess engine. This version was trained by learning from over 10 million Lichess games between 1100s. \r\n\r\nMaia Chess is an ongoing research project aiming to make a more human-friendly, useful, and fun chess AI. For more information go to maiachess.com. You can also play against @maia5 and @maia9. Developed by @ashtonanderson, @sidsen and @reidmcy.",
      "realName": "Maia Chess 1100",
      "links": "https://maiachess.com\r\nhttps://github.com/CSSLab/maia-chess\r\nhttps://twitter.com/maiachess"
    },
    "seenAt": 1758635494337,
    "playTime": {
      "total": 1083092110,
      "tv": 3019408
    }
  },
  {
    "id": "maia5",
    "username": "maia5",
    "perfs": {
      "bullet": {
        "games": 59096,
        "rating": 1569,
        "rd": 45,
        "prog": 17
      },
      "blitz": {
        "games": 161720,
        "rating": 1485,
        "rd": 45,
        "prog": -7
      },
      "rapid": {
        "games": 176792,
        "rating": 1640,
        "rd": 45,
        "prog": -14
      },
      "classical": {
        "games": 43066,
        "rating": 1621,
        "rd": 45,
        "prog": 8
      },
      "correspondence": {
        "games": 18,
        "rating": 1792,
        "rd": 228,
        "prog": 30,
        "prov": true
      }
    },
    "title": "BOT",
    "verified": true,
    "createdAt": 1582580198358,
    "profile": {
      "bio": "Maia is a human-like neural network chess engine. This version was trained by learning from over 10 million Lichess games between 1500s. Maia Chess is an ongoing research project aiming to make a more human-friendly, useful, and fun chess AI. For more information go to maiachess.com. You can also play @maia1 and @maia9. Developed by @ashtonanderson,  @sidsen and @reidmcy.",
      "realName": "Maia Chess 1500",
      "links": "https://maiachess.com\r\ngithub.com/CSSLab/maia-chess\r\nhttps://twitter.com/maiachess"
    },
    "seenAt": 1758635443518,
    "playTime": {
      "total": 508638915,
      "tv": 3849828
    }
  }
]